In [1]:
# python
import os
import math
import csv

# random
import random
#data analysis libraries
import numpy as np
import pandas as pd

# machine learning
import sklearn

# deep learning
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn.functional as F
import pytorch_lightning as pl

#d2l
import d2l.torch as d2l

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter


#visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# For Progress Bar
from tqdm import tqdm

#ignore warnings
import warnings

warnings.filterwarnings('ignore')

# auto load change
%load_ext autoreload
%autoreload 2

## Improvement
+ use parameter instead of hard-code for base class and task settings
+ metaR `batchnorm1d` is canceled by me

## Dataset set up
+ NELL one: 30 base classes, 21 novel classes(3 way 5 shot 7 sessions)

## Dataset and DataLoader
+ 基于 base class or novel class 数量构建 train_task.
use `next_batch` to get train_task shape: `(4, batch_size, num)`
+ every epoch will get same relations(`self.curr_rel_idx` will ensure this)
### construct support and query
random choose support(相同数量 positive and negative) query(相同数量 positive and negative) according to current relation
+ support query shape: `(batch_size, num, 2(e1_e2), embedding_shape)`

## MetaR train

### embedding entity
get embeddings according to entity id 

### relation learner
![](imgs/img.png)
![](imgs/img_4.png)

### gradient descent in support 
![](imgs/img_1.png)
![](imgs/img_2.png)
![](imgs/img_3.png)